In [3]:
import os
import requests
import base64

api_key="XXX"

IMAGE_PATH = "data/complex_report_screenshot.png"

encoded_image = base64.b64encode(open(IMAGE_PATH, 'rb').read()).decode('ascii')
headers = {
    "Content-Type": "application/json",
    "api-key": api_key,
}

# Payload for the request
payload = {
  "messages": [
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": "You are an AI assistant that extract information from images."
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{encoded_image}"
          }
        },
        {
          "type": "text",
          "text":"explain this image and give me launch date, fund size and risk class as key value pairs."
        }
      ]
    }
  ],
  "temperature": 0.7,
  "top_p": 0.95,
  "max_tokens": 800
}

endpoint = "https://aoai-eastus2-yk.openai.azure.com/openai/deployments/gpt-4turbo-0409/chat/completions?api-version=2024-02-15-preview"

# Send request
try:
    response = requests.post(endpoint, headers=headers, json=payload)
    response.raise_for_status()  # Will raise an HTTPError if the HTTP request returned an unsuccessful status code
except requests.RequestException as e:
    raise SystemExit(f"Failed to make the request. Error: {e}")

# Handle the response as needed (e.g., print or process)
print(response.json())

{'choices': [{'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}, 'finish_reason': 'stop', 'index': 0, 'message': {'content': 'This image is a detailed fund fact sheet for the "Principal - MPF Conservative Fund (CAP)". It provides comprehensive information about the fund including investment objectives, fund details, performance metrics, asset allocation, and the top ten holdings. The document is intended for investors to assess the characteristics and historical performance of the fund.\n\nHere are the requested key-value pairs from the fact sheet:\n\n- **Launch Date**: 1/12/2000\n- **Fund Size (in Million)**: HKD 3,804.49\n- **Risk Class**: Low (低)', 'role': 'assistant'}}], 'created': 1720058857, 'id': 'chatcmpl-9h6AjSs7o5XkHPXrA6NPbypjvJyhX', 'model': 'gpt-4-turbo-2024-04-09', 'object': 'chat.completion

In [4]:
import json
result = json.loads(response.text)["choices"][0]["message"]["content"]

In [5]:
result

'This image is a detailed fund fact sheet for the "Principal - MPF Conservative Fund (CAP)". It provides comprehensive information about the fund including investment objectives, fund details, performance metrics, asset allocation, and the top ten holdings. The document is intended for investors to assess the characteristics and historical performance of the fund.\n\nHere are the requested key-value pairs from the fact sheet:\n\n- **Launch Date**: 1/12/2000\n- **Fund Size (in Million)**: HKD 3,804.49\n- **Risk Class**: Low (低)'

# JSON FORMAT OUTPUT

In [6]:
import datetime
import json
import openai
import os
import sys

from openai import AzureOpenAI
from dotenv import load_dotenv
from pprint import pprint

In [7]:
client = AzureOpenAI(
    api_version="2024-02-15-preview",
    azure_endpoint="https://aoai-eastus2-yk.openai.azure.com/",
    api_key="XXX"
)

In [8]:
completion = client.chat.completions.create(
    model="gpt-4turbo-0409",
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant. Your response should be in JSON format.",
        },
        {
            "role": "user",
            "content": "List me 10 top movies from the 70's with the title, the director, the year \
        and the main actors",
        },
    ],
    response_format={"type": "json_object"},
)

In [9]:
print(completion)

ChatCompletion(id='chatcmpl-9h6BLjKEHuAp3EJMMh3GNozGL94Bv', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n  "movies": [\n    {\n      "title": "The Godfather",\n      "director": "Francis Ford Coppola",\n      "year": 1972,\n      "main_actors": ["Marlon Brando", "Al Pacino", "James Caan"]\n    },\n    {\n      "title": "Star Wars",\n      "director": "George Lucas",\n      "year": 1977,\n      "main_actors": ["Mark Hamill", "Harrison Ford", "Carrie Fisher"]\n    },\n    {\n      "title": "Jaws",\n      "director": "Steven Spielberg",\n      "year": 1975,\n      "main_actors": ["Roy Scheider", "Robert Shaw", "Richard Dreyfuss"]\n    },\n    {\n      "title": "One Flew Over the Cuckoo\'s Nest",\n      "director": "Miloš Forman",\n      "year": 1975,\n      "main_actors": ["Jack Nicholson", "Louise Fletcher", "Will Sampson"]\n    },\n    {\n      "title": "The Godfather Part II",\n      "director": "Francis Ford Coppola",\n      "

In [10]:
completion.usage

CompletionUsage(completion_tokens=550, prompt_tokens=50, total_tokens=600)

In [11]:
json_answer = json.loads(completion.choices[0].message.content)
pprint(json_answer)

{'movies': [{'director': 'Francis Ford Coppola',
             'main_actors': ['Marlon Brando', 'Al Pacino', 'James Caan'],
             'title': 'The Godfather',
             'year': 1972},
            {'director': 'George Lucas',
             'main_actors': ['Mark Hamill', 'Harrison Ford', 'Carrie Fisher'],
             'title': 'Star Wars',
             'year': 1977},
            {'director': 'Steven Spielberg',
             'main_actors': ['Roy Scheider', 'Robert Shaw', 'Richard Dreyfuss'],
             'title': 'Jaws',
             'year': 1975},
            {'director': 'Miloš Forman',
             'main_actors': ['Jack Nicholson',
                             'Louise Fletcher',
                             'Will Sampson'],
             'title': "One Flew Over the Cuckoo's Nest",
             'year': 1975},
            {'director': 'Francis Ford Coppola',
             'main_actors': ['Al Pacino', 'Robert Duvall', 'Diane Keaton'],
             'title': 'The Godfather Part II',
    

In [12]:
json_answer["movies"][0]

{'title': 'The Godfather',
 'director': 'Francis Ford Coppola',
 'year': 1972,
 'main_actors': ['Marlon Brando', 'Al Pacino', 'James Caan']}

In [13]:
for i in range(len(json_answer["movies"])):
    print(i+1, json_answer["movies"][i]["title"], json_answer["movies"][i]["year"])

1 The Godfather 1972
2 Star Wars 1977
3 Jaws 1975
4 One Flew Over the Cuckoo's Nest 1975
5 The Godfather Part II 1974
6 Apocalypse Now 1979
7 Taxi Driver 1976
8 A Clockwork Orange 1971
9 Rocky 1976
10 Chinatown 1974
